# CloudWatch

In [7]:
%%time
import boto3
import pandas as pd
from datetime import datetime, timedelta, timezone
import io
import json
from time import sleep
from threading import Thread

import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.huggingface.model import HuggingFaceModel

CPU times: user 573 µs, sys: 35 µs, total: 608 µs
Wall time: 618 µs


In [8]:
##S3 prefixes
bucket_name = 'aai-540-final-data-east-1'
region_name = 'us-east-1'
session = sagemaker.Session(boto3.Session(region_name=region_name))
s3 = session.boto_session.client('s3')
s3_path = 'athena/results/'

prefix = "sagemaker/AIEmotions-ModelQualityMonitor"
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"


ground_truth_upload_path = (
    f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region_name)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/datacapture
Ground truth path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-21-19-18-30
Report path: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/reports


In [4]:
'''
# Access Queried Test Data
bucket_name = 'aai-540-final-data'
s3_test_query = 'athena/results/f4cf6706-1ba6-4538-ab86-13c8137facb5.csv'
test_data_obj = s3.get_object(Bucket=bucket_name, Key = s3_test_query)
df_test = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))
'''

In [9]:
df_test_new = pd.read_csv('test.csv')
df_test_new

emotion_categories = {
	"anger": ["anger", "annoyance", "disapproval"],
	"disgust": ["disgust"],
	"fear": ["fear", "nervousness"],
	"happy": ["joy", "amusement", "approval", "gratitude"],
	"optimistic": ["optimism", "relief", "pride", "excitement"],
	"affectionate": [ "love", "caring", "admiration",  "desire"],
	"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
	"surprise": ["surprise", "realization", "confusion", "curiosity"],
	"neutral": ["neutral"]
} 

emotion_index_to_label = {index: label for index, label in enumerate(emotion_categories)}
df_test_new['emotions'] = df_test_new['emotions'].apply(lambda x: emotion_index_to_label[x])
df_test_new

,text,emotions
0,Yeah I tried to apply but with being partially...,sadness
1,"First time, health/environmental reasons. Seco...",neutral
2,Wow. I dig that.,affectionate
3,Thank you! We’re currently shoveling ourselves...,happy
4,This is one of my favorite of [NAME] songs alt...,affectionate
...,...,...
5422,"So, the kicker... her brother played on Xbox s...",neutral
5423,It would be hilarious if it was a pasta place too,happy
5424,"Oh damn, that would have been the only correct...",sadness
5425,"At this point, LSC only exists to give sociali...",anger


In [10]:
#Retrieve Model
role = sagemaker.get_execution_role()
s3 = boto3.client('s3')
model_dir = 'models/tuned_model/'
tar_file = 'model.tar.gz'


model_name = f"AIEmotion-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_data = f's3://{bucket_name}/{model_dir}{tar_file}'
tensorflow_version = '2.6.3'
transformers_version='4.17.0'
py_version = 'py38'
huggingface_model = HuggingFaceModel(model_data=model_data,
                                     role=role,
                                     transformers_version=transformers_version,
                                     tensorflow_version=tensorflow_version,
                                     py_version=py_version,
                                     entry_point='inference.py'
                                   )

In [11]:
endpoint_name = f"AIEmotion-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.c5.large',
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = AIEmotion-model-quality-monitor-2024-02-21-1918
-----!

In [12]:
# Predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket_name}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket_name}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data
Baseline results uri: s3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/results


In [13]:
# Grab a sample of the dataset
df_test_subset = df_test_new.sample(500)
df_test_subset = df_test_subset.reset_index(drop=True)

In [14]:
def calculate_predictions(results):
    df_results = pd.DataFrame(columns = ['pred_labels', 'probabilities'])
    for result in results:
        res = json.loads(result[0])
        pred_labels = list(res.keys())
        prob = list(res.values())
        row = {'pred_labels': pred_labels, 'probabilities': prob}
        df_results = df_results.append(row, ignore_index = True)
    return df_results

In [15]:
predictions = df_test_subset.apply(lambda x: predictor.predict({'text': x['text']}), axis = 1)
predictions

0      [{"affectionate": 0.5383890271186829, "happy":...
1      [{"neutral": 0.923974335193634, "happy": 0.029...
2      [{"neutral": 0.5757226943969727, "anger": 0.35...
3      [{"affectionate": 0.6554600596427917, "happy":...
4      [{"happy": 0.5222921967506409, "affectionate":...
                             ...                        
495    [{"happy": 0.9611902236938477, "affectionate":...
496    [{"anger": 0.9043861031532288, "neutral": 0.04...
497    [{"happy": 0.8997564315795898, "optimistic": 0...
498    [{"surprised": 0.7620328068733215, "neutral": ...
499    [{"anger": 0.8883174061775208, "neutral": 0.06...
Length: 500, dtype: object

In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df_preds = calculate_predictions(predictions)
df_preds

,pred_labels,probabilities
0,"[affectionate, happy, optimistic]","[0.5383890271186829, 0.38293197751045227, 0.04..."
1,"[neutral, happy, anger]","[0.923974335193634, 0.029579658061265945, 0.01..."
2,"[neutral, anger, disgust]","[0.5757226943969727, 0.35368630290031433, 0.02..."
3,"[affectionate, happy, optimistic]","[0.6554600596427917, 0.2711202800273895, 0.052..."
4,"[happy, affectionate, neutral]","[0.5222921967506409, 0.2225363403558731, 0.186..."
...,...,...
495,"[happy, affectionate, anger]","[0.9611902236938477, 0.01130503136664629, 0.00..."
496,"[anger, neutral, disgust]","[0.9043861031532288, 0.04557618126273155, 0.01..."
497,"[happy, optimistic, affectionate]","[0.8997564315795898, 0.04736463725566864, 0.02..."
498,"[surprised, neutral, happy]","[0.7620328068733215, 0.16513626277446747, 0.01..."


In [17]:
# Extract the first element of each tuple
df_preds['pred_labels'] = df_preds['pred_labels'].apply(lambda x: x[0])
df_preds['probabilities'] = df_preds['probabilities'].apply(lambda x: x[0])
df_preds

,pred_labels,probabilities
0,affectionate,0.538389
1,neutral,0.923974
2,neutral,0.575723
3,affectionate,0.655460
4,happy,0.522292
...,...,...
495,happy,0.961190
496,anger,0.904386
497,happy,0.899756
498,surprised,0.762033


In [18]:
# Concat datasets
validate_dataset = pd.concat([df_test_subset, df_preds], ignore_index=True, axis = 1)
validate_dataset = validate_dataset.rename(columns = {0: 'text', 
                                                      1: 'true_label', 
                                                      2: 'pred_label', 
                                                      3: 'probability'})

validate_dataset['pred_label'] = validate_dataset['pred_label'].replace({'sad': 'sadness', 'surprised': 'surprise'})
validate_dataset

,text,true_label,pred_label,probability
0,"Congrats, you got pleb filtered",affectionate,affectionate,0.538389
1,Admin is all over it!,neutral,neutral,0.923974
2,assault some more people.,neutral,neutral,0.575723
3,This actually made me squeal like a little gir...,affectionate,affectionate,0.655460
4,If I wasn't destitute I would give you gold. I...,happy,happy,0.522292
...,...,...,...,...
495,Ok thanks bro for the opinion. I’m trying to s...,happy,happy,0.961190
496,Fuck sake man,anger,anger,0.904386
497,I am just like this! Glad to know I’m not imag...,optimistic,happy,0.899756
498,Your girlfriend accepts gifts in exchange for ...,surprise,surprise,0.762033


In [19]:
# Export to CSV
validate_dataset_name = 'validate_dataset.csv'
validate_dataset.to_csv(validate_dataset_name, index=False) 

In [20]:
# Upload baseline
baseline_dataset_uri = S3Uploader.upload(f"{validate_dataset_name}", baseline_data_uri)
baseline_dataset_uri

's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

## Setup Baseline

In [21]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

baseline_dataset_uri = 's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

emotions_base_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

# Name of the model quality baseline job
baseline_job_name = f"AIEmotions-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"


# Execute the baseline suggestion job.
job = emotions_base_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="MulticlassClassification",
    inference_attribute="pred_label",
    probability_attribute="probability",
    ground_truth_attribute="true_label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name AIEmotions-model-baseline-job-2024-02-21-1924


.................................................................................!

In [22]:
baseline_job = emotions_base_model_quality_monitor.latest_baselining_job

In [23]:
multiclass_metrics = baseline_job.baseline_statistics().body_dict["multiclass_classification_metrics"]
pd.json_normalize(multiclass_metrics).T

,0
confusion_matrix._and._and,0.000000
"confusion_matrix._and._i'm_not_taking_any_risks""""_""",0.000000
confusion_matrix._and.disgust,0.000000
"confusion_matrix._and._abuse_and_brainwash_children_without_putting_their_liberal_noses_where_they_don't_belong.""""""",0.000000
confusion_matrix._and.surprise,0.000000
...,...
weighted_f0_5_best_constant_classifier.standard_deviation,0.001837
weighted_f1_best_constant_classifier.value,0.145052
weighted_f1_best_constant_classifier.standard_deviation,0.002212
weighted_f2_best_constant_classifier.value,0.212509


In [24]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["multiclass_classification_constraints"]).T

,threshold,comparison_operator
accuracy,0.682,LessThanThreshold
weighted_recall,0.682,LessThanThreshold
weighted_precision,0.677708,LessThanThreshold
weighted_f0_5,0.675832,LessThanThreshold
weighted_f1,0.675866,LessThanThreshold
weighted_f2,0.678707,LessThanThreshold


## Setup continuous model monitoring

In [25]:
# Generate a small sample from our testing data
synthetic_sample = df_test_new.sample(10)
synthetic_sample = synthetic_sample.reset_index(drop=True)
#synthetic_sample.to_csv('synthetic_data.csv', index=False)

In [26]:
predictions = synthetic_sample.apply(lambda x: predictor.predict({'text': x['text']}), axis = 1)
predictions

0    [{"anger": 0.5706148147583008, "surprised": 0....
1    [{"anger": 0.6091394424438477, "neutral": 0.20...
2    [{"neutral": 0.585700273513794, "anger": 0.301...
3    [{"anger": 0.8066050410270691, "surprised": 0....
4    [{"sad": 0.9122321009635925, "anger": 0.026297...
5    [{"anger": 0.5246332883834839, "neutral": 0.37...
6    [{"neutral": 0.5897254943847656, "surprised": ...
7    [{"happy": 0.6504971385002136, "affectionate":...
8    [{"surprised": 0.5263181924819946, "neutral": ...
9    [{"affectionate": 0.8705141544342041, "happy":...
dtype: object

In [27]:
preds = calculate_predictions(predictions)
preds

,pred_labels,probabilities
0,"[anger, surprised, neutral]","[0.5706148147583008, 0.18668246269226074, 0.13..."
1,"[anger, neutral, sad]","[0.6091394424438477, 0.20147308707237244, 0.07..."
2,"[neutral, anger, affectionate]","[0.585700273513794, 0.3018295466899872, 0.0253..."
3,"[anger, surprised, neutral]","[0.8066050410270691, 0.05767594277858734, 0.05..."
4,"[sad, anger, neutral]","[0.9122321009635925, 0.026297012344002724, 0.0..."
5,"[anger, neutral, sad]","[0.5246332883834839, 0.37060004472732544, 0.03..."
6,"[neutral, surprised, anger]","[0.5897254943847656, 0.36416923999786377, 0.01..."
7,"[happy, affectionate, neutral]","[0.6504971385002136, 0.19373853504657745, 0.08..."
8,"[surprised, neutral, anger]","[0.5263181924819946, 0.31863000988960266, 0.11..."
9,"[affectionate, happy, optimistic]","[0.8705141544342041, 0.06400194019079208, 0.04..."


In [28]:
preds['pred_labels'] = preds['pred_labels'].apply(lambda x: x[0])
preds['probabilities'] = preds['probabilities'].apply(lambda x: x[0])
preds

,pred_labels,probabilities
0,anger,0.570615
1,anger,0.609139
2,neutral,0.585700
3,anger,0.806605
4,sad,0.912232
5,anger,0.524633
6,neutral,0.589725
7,happy,0.650497
8,surprised,0.526318
9,affectionate,0.870514


In [29]:
%%time

import numpy as np
import time

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

#flat_list = []
for i in range(10):
    result = predictor.predict({'text': synthetic_sample['text'][i]})
    #flat_list.append(float("%.3f" % (np.array(result))))
    time.sleep(0.5)

print("Done!")
#print("predictions: \t{}".format(np.array(flat_list)))

Sending test traffic to the endpoint AIEmotion-model-quality-monitor-2024-02-21-1918. 
Please wait...
Done!
CPU times: user 30.2 ms, sys: 344 µs, total: 30.5 ms
Wall time: 7.52 s


In [30]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket_name, Prefix=data_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-1918/AllTraffic/2024/02/21/19/22-11-666-e235b121-fd0c-4af9-8bc9-881988992d84.jsonl
 sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-21-1918/AllTraffic/2024/02/21/19/23-11-769-244da09e-f334-460a-8f20-575c80bac8e3.jsonl


In [32]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket_name, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"I'd say [NAME].\"}","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[\n  \"{\\\"neutral\\\": 0.8980746269226074, \\\"happy\\\": 0.05805426836013794, \\\"surprised\\\": 0.010948991402983665}\",\n  \"application/json\"\n]","encoding":"JSON"}},"eventMetadata":{"eventId":"5a678e73-5f85-42bf-8a2f-e7e8758cc405","inferenceTime":"2024-02-21T19:23:11Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"Of all of the fan-favorites from each city, [NAME] being the fan-favorite confuses and annoys me most. Her headlines are becoming insufferable.\"}","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[\n  \"{\\\"anger\\\": 0.7004535794258118, \\\"neutral\\\": 0.07503578066825867, \\\"disgust\\\": 0.072432830

In [33]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "{\"text\": \"I'd say [NAME].\"}",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "[\n  \"{\\\"neutral\\\": 0.8980746269226074, \\\"happy\\\": 0.05805426836013794, \\\"surprised\\\": 0.010948991402983665}\",\n  \"application/json\"\n]",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "5a678e73-5f85-42bf-8a2f-e7e8758cc405",
    "inferenceTime": "2024-02-21T19:23:11Z"
  },
  "eventVersion": "0"
}


In [39]:
upload_time = datetime.utcnow()
target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
synthetic_sample['emotions'].to_json('output.jsonl', orient='records', lines=True)
S3Uploader.upload_string_as_file_body('output.jsonl', target_s3_uri)

's3://aai-540-final-data-east-1/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-21-19-18-30/2024/02/21/19/4758.jsonl'

In [45]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime
##Monitoring schedule name
model_monitor_schedule_name = (f"AIEmotion-model-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}")

endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint"
    )

emotions_base_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_monitor_schedule_name,
    endpoint_input=endpoint_input,
    output_s3_uri=s3_report_path,
    constraints=baseline_job.suggested_constraints(),
    ground_truth_input=ground_truth_upload_path,
    problem_type = 'MulticlassClassification',
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

ClientError: An error occurred (ValidationException) when calling the CreateModelQualityJobDefinition operation: InvalidParameter: 1 validation error(s) found.
- missing required field, CreateModelQualityJobDefinitionInput.ModelQualityJobInputs.EndpointInput.InferenceAttribute.


## Analyze Model Quality

## Release Resources

In [62]:
predictor.delete_endpoint()
predictor.delete_model()

INFO:sagemaker:Deleting endpoint configuration with name: AIEmotion-model-quality-monitor-2024-02-21-1751
INFO:sagemaker:Deleting endpoint with name: AIEmotion-model-quality-monitor-2024-02-21-1751


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_31/1204019056.py", line 2, in <module>
    predictor.delete_model()
  File "/usr/local/lib/python3.9/site-packages/sagemaker/predictor.py", line 339, in delete_model
  File "/usr/local/lib/python3.9/site-packages/sagemaker/predictor.py", line 530, in _get_model_names
  File "/usr/local/lib/python3.9/site-packages/botocore/client.py", line 530, in _api_call
    return op_dict
  File "/usr/local/lib/python3.9/site-packages/botocore/client.py", line 960, in _make_api_call
    # for use during construction.
botocore.exceptions.ClientError: An error occurred (ValidationException) when calling the DescribeEndpointConfig operation: Could not find endpoint configuration "AIEmotion-model-quality-monitor-2024-02-21-1751".

During handling of the above exception, another exception oc